In [1]:
import pandas as pd
import numpy as np
import os
import time
import ast
import re

os.listdir('../Data')

['BBDD_asturias_tourism', 'Data_used', 'Tourist_places']

In [2]:
alojamientos=pd.read_csv('../Data/Data_used/available_accommodation_data.csv')
restaurantes=pd.read_csv('../Data/Data_used/restaurant_data.csv')
turistico=pd.read_csv('../Data/Data_used/tourist_places_data.csv')
df_list=[turistico,alojamientos,restaurantes]
df_list_str=['turistico','alojamientos','restaurantes']
for ind,df in enumerate(df_list):
    print(df_list_str[ind])
    print(f'cols: {df.columns.to_list()}')
    print(f'shape: {df.shape}')

turistico
cols: ['Nombre', 'Localización', 'Latitud', 'Longitud', 'Características', 'Descripción', 'Tipo_lugar_turistico', 'url']
shape: (1101, 8)
alojamientos
cols: ['Nombre', 'Localización', 'Latitud', 'Longitud', 'Dirección', 'Valoración', 'Votos', 'Servicios', 'Descripción', 'url', 'estado', 'id_nombre', 'n_noches', 'Tipo_habitacion', 'Tipo_cama', 'Carac_habitacion', 'Número de personas', 'Precio', 'Carac_reserva']
shape: (2302, 19)
restaurantes
cols: ['Nombre', 'Localización', 'Latitud', 'Longitud', 'Dirección', 'Teléfono', 'Precio', 'Horario', 'Tipos_cocina', 'Valoración', 'Votos', 'Características', 'Descripción', 'url', 'url_web', 'url_ig']
shape: (2652, 16)


# Restaurantes

In [3]:
print(restaurantes.duplicated().sum())
print(restaurantes.url.duplicated().sum())
print()
for i, nom in enumerate(restaurantes.columns):
    print(i,nom)

5
33

0 Nombre
1 Localización
2 Latitud
3 Longitud
4 Dirección
5 Teléfono
6 Precio
7 Horario
8 Tipos_cocina
9 Valoración
10 Votos
11 Características
12 Descripción
13 url
14 url_web
15 url_ig


In [4]:
def url_ig_process(x):
    if x=='Desconocido':
        return x
    else:
        return 'https://www.instagram.com/'+x.replace('@','')+'/?hl=es'
def url_web_process(x):
    if x=='facebook.com':
        return 'Desconocido'
    elif x=='Desconocido':
        return x
    else:
        return 'https://'+x
def horario_process(x):
    if x=='Desconocido':
        return x
    elif x=='[]':
        return 'Desconocido'
    else:
        x=ast.literal_eval(x)
        if x[0]=='LunesLun':
            y=[pro for i,pro in enumerate(x) if i%2==1]
        else:
            y=[pro.split()[1] for pro in x]

        y=sep_horario(y)
    return y
def precio_process(x):
    if x=='Desconocido':
        return x
    else:
        x=x.strip().split('-')
        y=[pro.strip().replace('€','').replace('~hasta ','').replace('hasta ','').replace('más de ','').replace('~','') for pro in x]
    
    return y
def sep_horario(x):
    for i,cadena in enumerate(x):
        if len(cadena.split('-'))>2:
            #resultado = [cadena[i:i+11] for i in range(0, len(cadena), 11)]
            resultado = "/".join(cadena[i:i+11] for i in range(0, len(cadena), 11))
            x[i]=resultado
    return x
def restaurantes_process(df):
    '''
    FUNCION: Procesado basico del df restaurantes (eliminar duplicados y filas con KO, procesados especiales, paso a numeros y listas...)

    PARAMETROS:
        df: df de los datos de los restaurantes
    RETURN:
        restaurantes_pro: df de restaurantes procesado
    '''
    restaurantes_pro=df.copy()
    ################################    DUPLICADOS  ##########################
    restaurantes_pro=restaurantes_pro[restaurantes_pro['Nombre']!='KO']#eliminar los que fallaron del web-scraping
    restaurantes_pro=restaurantes_pro.drop_duplicates()
    ################################    PROCESAR  ##########################
    restaurantes_pro['url_ig']=restaurantes_pro['url_ig'].apply(url_ig_process)
    restaurantes_pro['url_web']=restaurantes_pro['url_web'].apply(url_web_process)
    restaurantes_pro['Horario']=restaurantes_pro['Horario'].apply(horario_process)
    restaurantes_pro['Precio']=restaurantes_pro['Precio'].apply(precio_process)
    ################################    PASAR A NUMEROS  ##########################
    restaurantes_pro['Valoración'] = restaurantes_pro['Valoración'].astype(int)
    restaurantes_pro['Votos'] = restaurantes_pro['Votos'].astype(int)
    restaurantes_pro['Latitud'] = restaurantes_pro['Latitud'].astype(float)
    restaurantes_pro['Longitud'] = restaurantes_pro['Longitud'].astype(float)
    ################################    PASAR A LISTAS  ##########################
    restaurantes_pro['Características']=restaurantes_pro['Características'].apply(lambda x: x if x=='Desconocido' else ast.literal_eval(x))
    restaurantes_pro['Tipos_cocina']=restaurantes_pro['Tipos_cocina'].apply(lambda x: x if x=='Desconocido' else list(set([pro.replace(',','').strip() for pro in ast.literal_eval(x)])))
    ################################    DESCONOCIDO-->X  ##########################
    restaurantes_pro=restaurantes_pro.replace('Desconocido','X')
    restaurantes_pro=restaurantes_pro.reset_index(drop=True)

    return restaurantes_pro

In [5]:
restaurantes_pro=restaurantes_process(restaurantes)
display(restaurantes_pro.head())
print(restaurantes_pro.dtypes)

,Nombre,Localización,Latitud,Longitud,Dirección,Teléfono,Precio,Horario,Tipos_cocina,Valoración,Votos,Características,Descripción,url,url_web,url_ig
0,Sidrería Narcea,Cangas del Narcea,43.178145,-6.551510,"C. Dos Amigos, Cangas del Narcea, Principado d...",+34985810038,"[21, 32]","[11:00-00:00, Cerrado, Cerrado, 11:00-00:00, 1...","[Española, Opciones vegetarianas]",90,5048,"[Asientos al aire libre, Para llevar, Tarjetas...",Visita Puente Colgante Cangas del Narcea y acé...,https://es.restaurantguru.com/Sidreria-Narcea-...,X,https://www.instagram.com/sidrerianarcea/?hl=es
1,Restaurante Casa del Río,Cangas del Narcea,43.178093,-6.550776,"C. Mayor, 38, Cangas del Narcea, Principado de...",+34985812356,"[10, 24]","[Cerrado, 08:00-01:00, 08:00-01:00, 08:00-01:0...","[Grill, Española, Opciones vegetarianas]",90,1591,"[Tarjetas de crédito aceptadas, Asientos al ai...","Primero, disfruta de Puente Colgante Cangas de...",https://es.restaurantguru.com/Restaurante-Casa...,https://restaurantecasadelrio.es,X
2,Bar Blanco Restaurante,Cangas del Narcea,43.176543,-6.549920,"C. Mayor, 11, Cangas del Narcea, Principado de...",+34985810316,"[10, 24]","[Cerrado, 10:30-16:00/19:00-01:30, 10:30-16:00...","[Española, Opciones vegetarianas]",88,1513,"[Asientos al aire libre, Tarjetas de crédito a...","Para un turista, tanto este restaurante como P...",https://es.restaurantguru.com/Bar-Blanco-Canga...,https://barblancorestaurante.es,https://www.instagram.com/barblancorest/?hl=es
3,Restaurante Leitariegos,Cangas del Narcea,42.998684,-6.418440,"Puerto de Leitariegos, s/n, Cangas del Narcea,...",+34985056201,"[19, 29]","[12:30-22:00, 12:30-22:00, Cerrado, 12:30-22:0...","[Española, Opciones vegetarianas]",86,507,"[Tarjetas de crédito aceptadas, Asientos al ai...","RESTAURANTE LEITERIEGOS, en un sitio privilegi...",https://es.restaurantguru.com/Restaurante-Leit...,https://restauranteleitariegos.com,https://www.instagram.com/restaurante_leitarie...
4,Parador de Corias,Cangas del Narcea,43.195484,-6.543462,"s/n, Monasterio de San Juan Bautista, Cangas d...",+34985070000,"[41, 69]","[13:30-16:00/20:30-23:00, 13:30-16:00/20:30-23...","[Opciones vegetarianas, Española, Saludable]",86,3020,"[Asientos al aire libre, Tarjetas de crédito a...",Pásate por aquí si eres un amante de las cocin...,https://es.restaurantguru.com/Restaurante-Espe...,https://parador.es,https://www.instagram.com/paradores/?hl=es


Nombre              object
Localización        object
Latitud            float64
Longitud           float64
Dirección           object
Teléfono            object
Precio              object
Horario             object
Tipos_cocina        object
Valoración           int64
Votos                int64
Características     object
Descripción         object
url                 object
url_web             object
url_ig              object
dtype: object


# Alojamientos

In [6]:
print(alojamientos.duplicated().sum())
print(alojamientos.url.duplicated().sum())
print()
for i, nom in enumerate(alojamientos.columns):
    print(i,nom)

11
15

0 Nombre
1 Localización
2 Latitud
3 Longitud
4 Dirección
5 Valoración
6 Votos
7 Servicios
8 Descripción
9 url
10 estado
11 id_nombre
12 n_noches
13 Tipo_habitacion
14 Tipo_cama
15 Carac_habitacion
16 Número de personas
17 Precio
18 Carac_reserva


In [7]:
def valoracion_procces2(x):
    if x == "Desconocido":
        x = 0
    return x

def alojamientos_process(df):
    '''
    FUNCION: Procesado basico del df restaurantes (eliminar duplicados y filas con KO, procesados especiales, paso a numeros y listas...)

    PARAMETROS:
        df: df de los datos de los restaurantes
    RETURN:
        restaurantes_pro: df de restaurantes procesado
    '''
    alojamientos_pro=df.copy()
    ################################    DUPLICADOS  ##########################
    alojamientos_pro=alojamientos_pro[alojamientos_pro['Precio'] != "[]" ] #Eliminar los que fallaron del web-scraping
    alojamientos_pro=alojamientos_pro.drop_duplicates() # Eliminar duplicados
    alojamientos_pro = alojamientos_pro.drop(columns=["estado", "id_nombre"]) # Eliminar variables innnecesarias
    # ################################    PROCESAR  ##########################
    alojamientos_pro["Valoración"] = alojamientos_pro["Valoración"].apply(valoracion_procces2)
    # ################################    PASAR A NUMEROS  ##########################
    alojamientos_pro['Valoración'] = alojamientos_pro['Valoración'].astype(float)
    alojamientos_pro['Votos'] = alojamientos_pro['Votos'].astype(int)
    alojamientos_pro['Latitud'] = alojamientos_pro['Latitud'].astype(float)
    alojamientos_pro['Longitud'] = alojamientos_pro['Longitud'].astype(float)
    # ################################    PASAR A LISTAS  ##########################
    alojamientos_pro["Servicios"] = alojamientos_pro["Servicios"].apply(lambda x: list(set(x)) if isinstance(x, list) else x)
    # ################################    DESCONOCIDO-->X  ##########################
    alojamientos_pro=alojamientos_pro.replace('Desconocido','X')
    alojamientos_pro=alojamientos_pro.reset_index(drop=True)

    return alojamientos_pro

In [8]:
alojamientos_pro=alojamientos_process(alojamientos)
display(alojamientos_pro.head())
print(alojamientos_pro.dtypes)

,Nombre,Localización,Latitud,Longitud,Dirección,Valoración,Votos,Servicios,Descripción,url,n_noches,Tipo_habitacion,Tipo_cama,Carac_habitacion,Número de personas,Precio,Carac_reserva
0,Hotel Nueva Allandesa,Pola de Allande,43.271429,-6.609451,"Donato Fernández, 3, 33880 Pola de Allande, Es...",7.9,1301,"['WiFi gratis ', 'Habitaciones sin humo ', 'Re...",El Hotel Nueva Allandesa se encuentra en el ce...,https://www.booking.com/hotel/es/nueva-allande...,1,"['Habitación Doble - 1 o 2 camas', 'Habitación...","['2 camas individuales', '1 cama individual']","[['Vistas a la montaña', 'Vistas al río', 'Bañ...","[2, 1]","[65, 40]",[['Cancelación gratis antes del 18 de febrero ...
1,Casa Gayón,Pola de Allande,43.273815,-6.619590,"El Mazo, 4, 33890 Pola de Allande, España",9.2,138,"['WiFi gratis ', 'Habitaciones sin humo ', 'Pa...","Casa Gayón, que cuenta con jardín, terraza y r...",https://www.booking.com/hotel/es/casa-gaya3n.e...,2,"['Apartamento con vistas a las montañas', 'Apa...","['Dormitorio\xa01:1 cama dobley1 sofá cama', '...","[['Apartamento entero', '60 m²', 'Cocina priva...","[2, 4]","[160, 200]","[['Pago de la 1ª noche por cancelar', 'Sin pag..."
2,Apartamentos Rurales Casa Carmen,Pola de Allande,43.232202,-6.563130,"Puente de linares s/n, 33890 Pola de Allande, ...",9.4,89,"['WiFi gratis ', 'Habitaciones sin humo ', 'Pa...",Apartamentos Rurales Casa Carmen está en Pola ...,https://www.booking.com/hotel/es/apartamentos-...,2,"['Apartamento de 2 dormitorios', 'Apartamento ...","['Dormitorio\xa01:1 cama doble', 'Dormitorio\x...","[['Apartamento entero', '48 m²', 'Cocina priva...","[4, 4]","[220, 240]",[['Cancelación gratis antes del 15 de febrero ...
3,Casa el rey 12 - Apartamentos Buenos Aires y F...,Valbona,43.262196,-6.576329,"Casa el rey, 12 33889 Figueras Allande Asturia...",9.3,54,['Parking gratis '],Casa el rey 12 - Apartamentos Buenos Aires y F...,https://www.booking.com/hotel/es/casa-el-rey-1...,1,"['Apartamento de 1 dormitorio', 'Apartamento d...","['1 cama doble', '1 cama doble']","[['Apartamento entero', '28 m²', 'Zona de coci...","[2, 2]","[79, 87]","[['No reembolsable', '• Pagas online'], ['Pa..."
4,Albergue Los Hospitales,Colinas de Arriba,43.313556,-6.592020,"Colinas de Arriba, Spain, 33878 Colinas de Arr...",9.3,334,"['Parking gratis ', 'Bar ', 'Desayuno ']","Albergue Los Hospitales, en Colinas de Arriba,...",https://www.booking.com/hotel/es/albergue-los-...,1,['Cama Individual en dormitorio compartido de ...,['1 cama individual'],"[['Vistas al jardín', 'Vistas a la montaña', '...",[1],[21],"[['Desayuno €\xa05 (opcional)', 'Cancelación ..."


Nombre                 object
Localización           object
Latitud               float64
Longitud              float64
Dirección              object
Valoración            float64
Votos                   int64
Servicios              object
Descripción            object
url                    object
n_noches                int64
Tipo_habitacion        object
Tipo_cama              object
Carac_habitacion       object
Número de personas     object
Precio                 object
Carac_reserva          object
dtype: object


# Lugares turísticos

In [9]:
print(turistico.duplicated().sum())
print(turistico.url.dropna().duplicated().sum())
print()
for i, nom in enumerate(turistico.columns):
    print(i,nom)

205
0

0 Nombre
1 Localización
2 Latitud
3 Longitud
4 Características
5 Descripción
6 Tipo_lugar_turistico
7 url


In [10]:
def list_process(lst):
    palabras_prohibidas = ["Trazado de la ruta", "Descarga", "Documentos", "Callejero"]
    return [x.replace('\n', '').replace('\t', '').replace('■', '').replace(" |", ",") # Cambiar valores "raros" 
            for x in lst 
            if not any(p in x for p in palabras_prohibidas) # Eliminar registros que tenía imágenese, documentos, ...
            and not x.strip().endswith(":")
            and ":" in x]

def caract_process(x):  # Procesar las listas de características
    if isinstance(x, str) and x.startswith("["):  # Evitar errores con valores no string
        try:
            lista = ast.literal_eval(x)
            return list_process(lista)
        except:
            return []  # En caso de error, devolver lista vacía
    return x 

def text_process(texto):
    
    try: 
        texto = re.sub(r'(?<=[a-záéíóúñ])([A-Z])', r'. \1', texto) # Añadir un punto antes de cada palabra en mayúscula si no hay puntuación antes.
        texto = re.sub(r'(?<!\d)\.(?!\d)', '. ', texto) # Asegurar que los puntos tengan un espacio después (evita casos con números decimales).
        texto = re.sub(r'\s+', ' ', texto).strip() # Limpiar dobles espacios.
    except:
        pass

    return texto

def turistico_process(df):
    '''
    FUNCION: Procesado basico del df lugares turísticos ( procesados especiales, paso a numeros y listas...)

    PARAMETROS:
        df: df de los datos de los lugares turísticos
    RETURN:
        turistico_pro: df de lugares turísticos procesado
    '''
    turistico_pro=df.copy()
    ################################    PROCESAR  ##########################
    turistico_pro = turistico_pro.dropna(subset=['Latitud', 'Longitud'])
    turistico_pro = turistico_pro.drop_duplicates(subset=["Nombre"], keep='first')
    ################################    PASAR A NUMEROS  ##########################
    turistico_pro['Latitud'] = turistico_pro['Latitud'].astype(float)
    turistico_pro['Longitud'] = turistico_pro['Longitud'].astype(float)
    ################################    PASAR A LISTAS  ##########################
    turistico_pro['Características']=turistico_pro['Características'].apply(caract_process).apply(lambda x: 'X' if x == [] else x)
    ################################    PROCESAR TEXTO  ##########################
    turistico_pro["Descripción"] = turistico_pro["Descripción"].apply(text_process)
    ################################    PROCESAR ERRORES  ##########################
    turistico_pro.loc[turistico_pro["Longitud"] == -6794391.0, "Longitud"] = -6.794391
    ################################    DESCONOCIDO-->X  ##########################
    turistico_pro = turistico_pro.replace('[]','X')
    turistico_pro = turistico_pro.reset_index(drop=True)

    return turistico_pro

In [11]:
turistico_pro=turistico_process(turistico)
display(turistico_pro.sample(5))
print(turistico_pro.dtypes)

,Nombre,Localización,Latitud,Longitud,Características,Descripción,Tipo_lugar_turistico,url
592,Ruta de las Brañas Vaqueiras en Cudillero,Cudillero,43.520384,-6.220032,"[Distancia:23,9kms, Dificultad:Media, Código:P...",Iglesia de San Martín de Luiña - Iglesia de Sa...,Ruta,https://www.turismoasturias.es/descubre/natura...
634,Ruta Pravia a Santianes,Pravia,43.490337,-6.106132,"[Distancia:4,9kms, Dificultad:Baja, Código:PR2...",CORTADA TEMPORALMENTERiqueza es lo que produce...,Ruta,https://www.turismoasturias.es/descubre/natura...
262,Collanzo,Aller,43.122407,-5.560932,X,El área se encuentra a la salida de la localid...,Naturaleza,https://www.turismoasturias.es/descubre/natura...
451,Santa María del Naranco,Oviedo,43.379055,-5.865849,[Dirección:Avda de los Monumentos (Monte Naran...,El Palacio de Ramiro I ubicado en la falda del...,Prerromanico,https://www.turismoasturias.es/descubre/cultur...
277,Monasterio e Iglesia de San Antolín de Bedón,Llanes,43.438146,-4.869111,"[Dirección:33594 - Bricia, Teléfono:626 990 39...","Se desconoce su año de construcción, ya que la...",Patrimonio religioso,https://www.turismoasturias.es/descubre/cultur...


Nombre                   object
Localización             object
Latitud                 float64
Longitud                float64
Características          object
Descripción              object
Tipo_lugar_turistico     object
url                      object
dtype: object


# Guardado

In [12]:
restaurantes_pro.to_csv("../Scripts/Definitive_data/restaurant_processed.csv", index=False)
alojamientos_pro.to_csv("../Scripts/Definitive_data/accommodation_processed.csv", index=False)
turistico_pro.to_csv("../Scripts/Definitive_data/tourist_processed.csv", index=False)
